In [1]:
# --- Instala as bibliotecas necessárias ---
# Usamos '-q' para uma instalação mais "silenciosa"
!pip install ultralytics roboflow -q

# --- Importa as bibliotecas ---
import os
from google.colab import drive
from ultralytics import YOLO
from roboflow import Roboflow

# --- Monta o Google Drive (irá pedir autorização) ---
print("Conectando ao Google Drive...")
drive.mount('/content/drive')
print("Google Drive conectado com sucesso!")

# --- VERIFICAÇÃO ---
# Vamos garantir que a conexão funcionou
!ls -l /content/drive/MyDrive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 46.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Conectando ao Google Drive...
Mounted at /content/drive
Google Drive conectado com sucesso!
total 6602429
drwx------ 2 root root       4096 Jan 26  2018 '$$$'
-rw------- 1 root root     542044 Jul 24  2014  0710_10.jpg
-rw-

In [2]:
# Célula Combinada: Passo 2 e 3 - Configuração e Download do Dataset (Versão 5)

# --- Suas Informações (ATUALIZADAS) ---
ROBOFLOW_API_KEY = "TzRI56Dfppr7HH6WksTx"       # Mantido do seu código
ROBOFLOW_WORKSPACE_ID = "verdpoc"               # Mantido do seu código
ROBOFLOW_PROJECT_ID = "label_verd-svjln"        # Mantido do seu código
ROBOFLOW_VERSION_NUMBER = 5                     # ATUALIZADO para a nova versão

# --- Configuração do Google Drive (sugiro um novo nome para o experimento) ---
GDRIVE_PROJECT_PATH = '/content/drive/MyDrive/Tese_Mestrado/With_annotation/Models' # Pasta principal de treinos
EXPERIMENT_NAME = 'Experiment_Product_5class_b' #project name

# Cria o diretório no Drive para salvar os checkpoints
caminho_experimento_completo = os.path.join(GDRIVE_PROJECT_PATH, EXPERIMENT_NAME)
os.makedirs(caminho_experimento_completo, exist_ok=True)
print(f"Diretórios prontos. Os resultados serão salvos em: {caminho_experimento_completo}")


# --- Conecta ao Roboflow e baixa o dataset COM A CORREÇÃO ---
try:
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace(ROBOFLOW_WORKSPACE_ID).project(ROBOFLOW_PROJECT_ID)

    # Usando o número da versão que você especificou
    version = project.version(ROBOFLOW_VERSION_NUMBER)

    # AQUI ESTÁ A CORREÇÃO: Usamos "yolov8" em vez de "yolov8-obb" para compatibilidade
    dataset = version.download("yolov8")

    print("\nDownload do dataset (versão 4) concluído com sucesso!")

    # --- VERIFICAÇÃO ---
    # Garante que o arquivo data.yaml foi baixado corretamente
    print("\nVerificando conteúdo da pasta do dataset:")
    !ls -l {dataset.location}

except Exception as e:
    print(f"\nOcorreu um erro ao baixar o dataset: {e}")
    print(f"Por favor, verifique se a versão {ROBOFLOW_VERSION_NUMBER} do seu projeto foi gerada com sucesso no Roboflow.")


Diretórios prontos. Os resultados serão salvos em: /content/drive/MyDrive/Tese_Mestrado/With_annotation/Models/Experiment_Product_5class_b
loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to Label_VERD-5 in yolov8:: 100%|██████████| 25984/25984 [00:03<00:00, 7667.01it/s] 



Download do dataset (versão 4) concluído com sucesso!

Verificando conteúdo da pasta do dataset:
total 24
-rw-r--r-- 1 root root  345 Oct 20 02:26 data.yaml
-rw-r--r-- 1 root root  307 Oct 20 02:26 README.dataset.txt
-rw-r--r-- 1 root root 1573 Oct 20 02:26 README.roboflow.txt
drwxr-xr-x 4 root root 4096 Oct 20 02:26 test
drwxr-xr-x 4 root root 4096 Oct 20 02:26 train
drwxr-xr-x 4 root root 4096 Oct 20 02:26 valid


In [3]:
# --- Carrega um modelo YOLOv8 pré-treinado ---
model = YOLO('yolov8n.pt')

# --- Inicia o treinamento ---
print("\nIniciando o treinamento do YOLOv8...")
print(f"Os resultados serão salvos em tempo real na pasta: {caminho_experimento_completo}")

results = model.train(
    data=f'{dataset.location}/data.yaml',  # Usa o caminho dinâmico do dataset baixado
    epochs=50,                             # Treina por 50 épocas
    imgsz=640,                             # Tamanho padrão da imagem
    project=GDRIVE_PROJECT_PATH,           # Pasta principal no Drive para salvar
    name=EXPERIMENT_NAME,                  # Subpasta para este experimento específico
    exist_ok=True                          # Permite retomar treinos interrompidos
)

print("\n--- TREINAMENTO CONCLUÍDO! ---")

# --- VERIFICAÇÃO FINAL ---
print("\nVerificando se os modelos foram salvos no Google Drive:")
!ls -l "{caminho_experimento_completo}/weights/"


Iniciando o treinamento do YOLOv8...
Os resultados serão salvos em tempo real na pasta: /content/drive/MyDrive/Tese_Mestrado/With_annotation/Models/Experiment_Product_5class_b
Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Label_VERD-5/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.